# Cartelera de hoy

+ ### Aritmética de punto flotante
+ ### Redondeos
+ ### Errores


## Aritmética de punto flotante (continuación)

Hasta ahora hemos visto que hay varios aspectos que inciden en la práctica con el uso de la APF

<ul>
    <li> <font color='blue'>Matemáticas:</font> hay aspectos que no son completamente trasladables a la APF (racionales con expresión infinita, irracionales, propiedades de los reales, etc.).
    <li> <font color='blue'>APF:</font> tiene sus características propias implementadas en el estándar IEEE 754-2008.
    <li> <font color='blue'>Python:</font> aunque Python implementa el estándar IEEE 754-2008, hay elementos sobre los cuales es necesario estar advertidos. 
</ul>

## Ejemplo de esta combinación de aspectos

$0.1_{10} = 0.0\overline{0011}_2$

$0.2_{10} = 0.\overline{0011}_2$

$0.1_{10} + 0.2_{10} = 0.3_{10}$

<font color='red'>¿Qué sucede si hacemos las operaciones en base 2?</font>

In [ ]:
a = 0.1
b = 0.2
a + b

0.30000000000000004

In [ ]:
import math

a = math.sqrt(2)
2.0 == a*a

False

In [ ]:
# usando Numpy
import numpy as np

a = np.float32(0.1)
b = np.float32(0.2)
a + b

In [ ]:
# usando Numpy
import numpy as np

a = np.float32(0.1)
b = np.float32(0.2)
print(a + b)

print('%.17f' % a)
print('%.17f' % b)
print('%.17f' % (a + b))

0.3
0.10000000149011612
0.20000000298023224
0.30000001192092896


## El estándar IEEE 754 - 2008

<br>
<div>
<img src="attachment:ieee-formato_flotante.png" width="800"/>
</div>

<div>
<img src="attachment:Single-Precision-IEEE-754-Floating-Point-Standard.jpg" width="500"/>
</div>

<div>
<img src="attachment:Double-Precision-IEEE-754-Floating-Point-Standard-1024x266.jpg" width="700"/>
</div>

### Ejemplo para 0.1

$0.1_{10} = 0.0\overline{0011}_2$

En notación normalizada

$0.0\overline{0011}_2 = 1.1001\overline{1001}\times2^{-4}$

+ Bit para el signo $\to 0$ 

+ Puesto que solo se tienen 8 bits para los exponentes, los 256 valores se deben utilizar para representar exponentes positivos y negativos, por lo que se usa un *biased exponent*, en 32 bits el *bias* es 127, de modo que valores < 127 corresponden a exponentes negativos, 127 a cero y valores > 127 a exponentes positivos. En este caso el *biased exponent* es 127 - 4 = 123. 

+ $123_{10} = ?_2$

#### Calculado la mantisa

+ La parte entera (1) no se guarda, se denomina el bit escondido (*hidden bit*)

+ En el caso de precisión sencilla se utilizan 23 bits; por tanto, la parte decimal de $1.1001\overline{1001}_2$ se trunca en 23 bits: $1001 1001 1001 1001 1001 100$.

+ Sin embargo, el valor al calcular $2^{bit\_24} > 0$, por lo que el bit 23 se redondea a 1. De esta manera los 23 bits quedan como $1001 1001 1001 1001 1001 101$.

#### Agrupando los bits calculados

$0$ &nbsp;&nbsp; $01111011$ &nbsp;&nbsp; $1001 1001 1001 1001 1001 101$

In [ ]:
# obteniendo la respresentación en bits de 0.1
import bitstring

f = bitstring.BitArray(float=0.1, length=32)
f.bin

'00111101110011001100110011001101'

#### Recalculando el valor decimal de la representación en base 2

+ $1.10011001100110011001101\times2^{-4} = 0.000110011001100110011001101$
+ Los exponentes son -4, -5, -8, -9, -12, -13, -16, -17, -20, -21, -24, -25, -27

In [ ]:
import numpy as np

LE = [-4, -5, -8, -9, -12, -13, -16, -17, -20, -21, -24, -25, -27]
suma = 0.0

for k in LE:
    suma += 2**k
    
print('valor calculado: ', suma)
print('valor por numpy:  %.17f' % np.float32(0.1))

valor calculado:  0.10000000149011612
valor por numpy:  0.10000000149011612


#### Casos especiales

+ Puesto que las secuencias de bits normalizadas incluyen un 1.
+ Existen secuencias especiales de bits para representar números 'especiales' como el cero, en el que todos los bits son 0.
+ $\pm$ infinito, todos los bits del exponente son 1 y la mantisa solo tienen ceros.
+ NaN, 'Not a Number' generalmente cuando no está definido por alguna razón o no es representable.